## Message Translator (FunctionalTransformer from YAML spec)

This notebook demonstrates the Message Translator pattern using a `FunctionalTransformer` defined in a YAML guild specification. Unlike PayloadTransformer which only transforms the payload, FunctionalTransformer can modify the entire MessageRoutable including routing information.

**Input Format**: `OrderRequest` - An e-commerce order  
**Output Format**: `InventoryCheck` - An inventory validation request

**YAML Transformation Benefits:**
- **Declarative Configuration**: Guild specification is defined in YAML
- **Variable Assignment**: Uses JSONata variable assignments for better readability
- **Dynamic Routing**: Routes messages based on calculated order totals
- **Complex Logic**: Handles both payload transformation and routing decisions
- **Reusable Variables**: Intermediate calculations broken into named components

The YAML specification includes a structured JSONata expression that:
1. Assigns intermediate variables for ID generation and calculations
2. Calculates order totals and determines priority levels
3. Maps order items to inventory check format
4. Makes dynamic routing decisions based on order value
5. Builds the complete MessageRoutable structure

The flow will be:

[ProbeAgent] --`OrderRequest`-> (default_topic) -> [Translator Agent/TranslatorAgent:wire_message] --`InventoryCheck`-> (inventory_high_priority | inventory_normal)

The routing destination is determined based on the order total during translation.

In [ ]:
from pydantic import BaseModel
from typing import List

# Input message format - e-commerce order
class OrderItem(BaseModel):
    product_id: str
    quantity: int
    unit_price: float

class OrderRequest(BaseModel):
    order_id: str
    customer_id: str
    items: List[OrderItem]
    shipping_address: str

# Output message format - inventory check
class InventoryItem(BaseModel):
    sku: str
    requested_quantity: int
    
class InventoryCheck(BaseModel):
    check_id: str
    order_reference: str
    customer_reference: str
    items_to_check: List[InventoryItem]
    total_value: float
    priority_level: str

In [ ]:
from rustic_ai.core.guild.builders import GuildBuilder

# Load the guild from YAML specification
guild = GuildBuilder.from_yaml_file("./005_message_translator_functional.yaml").launch("myorg")

In [ ]:
from rustic_ai.core.agents.testutils.probe_agent import ProbeAgent
from rustic_ai.core.guild.builders import AgentBuilder
from rustic_ai.core.guild.dsl import GuildTopics

guild_default_topic = GuildTopics.DEFAULT_TOPICS[0]

probe_agent = (
    AgentBuilder(ProbeAgent)
    .set_id("ProbeAgent")
    .set_name("Probe Agent")
    .set_description("A probe agent to test message translation with dynamic routing.")
    .add_additional_topic("inventory_high_priority")
    .add_additional_topic("inventory_normal")
    .build()
)

guild._add_local_agent(probe_agent)

In [ ]:
# Send a high-value order (should route to inventory_high_priority)
high_value_order = OrderRequest(
    order_id="ORD-001",
    customer_id="CUST-12345",
    items=[
        OrderItem(product_id="LAPTOP-001", quantity=2, unit_price=899.99),  # $1799.98
        OrderItem(product_id="MOUSE-001", quantity=2, unit_price=49.99)     # $99.98
    ],  # Total: $1899.96 (> $1000, should be high priority)
    shipping_address="123 Main St, City, State 12345"
)

# Calculate total for demonstration
total_value = sum(item.quantity * item.unit_price for item in high_value_order.items)
print(f"High-value order total: ${total_value:.2f} (should route to high priority)")

probe_agent.publish_with_guild_route(
    payload=high_value_order,
    topic=guild_default_topic,
)

In [ ]:
# Check the translated message and routing
probe_agent.print_message_history()

In [ ]:
# Send a low-value order (should route to inventory_normal)
low_value_order = OrderRequest(
    order_id="ORD-002", 
    customer_id="CUST-67890",
    items=[
        OrderItem(product_id="BOOK-001", quantity=3, unit_price=15.99),    # $47.97
        OrderItem(product_id="PEN-001", quantity=5, unit_price=2.50)       # $12.50
    ],  # Total: $60.47 (< $1000, should be normal priority)
    shipping_address="456 Oak Ave, Town, State 67890"
)

# Calculate total for demonstration
total_value = sum(item.quantity * item.unit_price for item in low_value_order.items)
print(f"Low-value order total: ${total_value:.2f} (should route to normal priority)")

probe_agent.publish_with_guild_route(
    payload=low_value_order,
    topic=guild_default_topic,
)

In [ ]:
# Check all messages to see both translations and different routing
probe_agent.print_message_history()

In [ ]:
# Inspect the last two messages to see the transformation and routing
messages = probe_agent.get_messages()
print("Message Analysis:")
print("=" * 50)

for i, msg in enumerate(messages[-2:], 1):
    print(f"\nMessage {i}:")
    print(f"  Format: {msg.format}")
    print(f"  Topic: {msg.topic_published_to}")
    print(f"  Payload Type: {type(msg.payload).__name__}")
    if hasattr(msg.payload, 'total_value'):
        print(f"  Total Value: ${msg.payload.total_value}")
        print(f"  Priority Level: {msg.payload.priority_level}")
        print(f"  Check ID: {msg.payload.check_id}")
    print(f"  Routing Decision: {'High Priority' if 'high_priority' in msg.topic_published_to else 'Normal Priority'}")